# This notebook contains the experiments on Banknote dataset with LionForests

In [1]:
import sys
cpath = !pwd
sys.path.append(cpath[0][:-43]) 
sys.path.append(cpath[0][:-43]+'/algorithms')

In [2]:
from lionforests import LionForests
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.DefragTrees.defragTrees import DefragModel
from CHIRPS.structures import data_container
import CHIRPS.routines as rt
import CHIRPS.structures as strcts

from scipy import sparse

from datasets.dataset import Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
np.seterr(invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import time

from sklearn.model_selection import train_test_split, LeaveOneOut, KFold
from sklearn.metrics import f1_score, precision_score

from anchor import anchor_tabular


Firstly, we load the dataset and we set the feature and class names

In [3]:
banknotes = Dataset()
X, y, feature_names, class_names = banknotes.load_banknote()
#one_h_percent = int(min(10*len(X)/100,100))
#print("Instances:",one_h_percent)
new_fn = []
for i in feature_names:
    new_fn.append(i.replace(' ','_'))
feature_names = new_fn

In this set of experiments we are using this not very well written code to perform few experiments/comparisons though different datasets and algorithms!

In [4]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names, tech=False):
    parameters = [{
        'max_depth': [10],
        'max_features': [0.75],
        'bootstrap': [True],
        'min_samples_leaf' : [1],
        'n_estimators': [500]
    }]
    scaler = MinMaxScaler(feature_range=(-1,1))
    lf = LionForests(None, False, scaler, feature_names, class_names)
    lf.fit(X_train, y_train, params=parameters)
    
    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)
    
    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task):

        #BaselineTechnique ==============================================================================
        gt = GlobalSurrogateTree(train, predictions, feature_names, task)
        lt = LocalSurrogateTree(train, predictions, feature_names, task, 150)

        #DefragTechnique ================================================================================
        Kmax = 10
        splitter = DefragModel.parseSLtrees(model) # parse sklearn tree ensembles into the array of (feature index, threshold)
        mdl = DefragModel(modeltype=task, maxitr=100, qitr=0, tol=1e-6, restart=20, verbose=0)
        mdl.fit(train, predictions, splitter, Kmax, fittype='FAB', featurename=feature_names)
        def def_cov(instances):
            ts = time.time()
            score, cover, coll = mdl.evaluate(instances[1:],lf.model.predict(instances[1:]))
            def_predictions = mdl.predict(instances[1:])
            rule = mdl.find_rule_length(instances[0])
            te = time.time()
            #print(mdl)
            return rule, cover, def_predictions, te-ts

        #Anchors =======================================================================================
        explainer = anchor_tabular.AnchorTabularExplainer(class_names, feature_names, train)
        def anchors_method(instance):
            exp = explainer.explain_instance(instance, lf.model.predict, threshold=0.95)
            anchors_dict = {}
            for i in exp.names():
                terms = i.split(' ')
                if len(terms) == 3:
                    anchors_dict[terms[0]] = [[terms[1],float(terms[2])]]
                else:
                    anchors_dict[terms[2]] = [[terms[3],float(terms[4])],[terms[1],float(terms[0])]]
            return anchors_dict, 0

        #CHIRPS =======================================================================================
        project_dir = cpath[0][:-43]+'/algorithms/CHIRPS'
        temp_frame = pd.DataFrame(np.hstack((train,y_train.reshape(len(y_train),1))),columns=feature_names+['class'])
        temp_frame['class']=temp_frame['class'].astype(int)

        temp_frame = temp_frame.replace({"class": {1: 2}})
        temp_frame = temp_frame.replace({"class": {0: 1}})

        mydata = data_container(
                data = temp_frame, class_col = 'class', var_names = ['variance', 'skew', 'curtosis', 'entropy'],
                project_dir = project_dir, save_dir = 'banknote', random_state=123)
        meta_data = mydata.get_meta()
        f_walker = strcts.classification_trees_walker(forest=model, meta_data=meta_data)
        f_walker.forest_walk(instances = test, labels = model.predict(test), forest_walk_async = True)

        explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                        forest=model,
                                        sample_instances=sparse.csr_matrix(train), # any representative sample can be used
                                        sample_labels=predictions,
                                        meta_data=meta_data)
        chts = time.time()
        explanations.run_explanations(target_classes=model.predict(test), # we're explaining the prediction, not the true label!
                                explanation_async=False,
                                random_state=123,
                                which_trees='majority',
                                alpha_paths=0.0,
                                support_paths=0.1,
                                score_func=1,
                                precis_threshold=0.99,
                                disc_path_bins=4,
                                merging_bootstraps=20,
                                pruning_bootstraps=20,
                                delta=0.2,
                                weighting='kldiv')
        chte = (time.time() - chts)/len(test)

        def chirps_method(idx):
            chirps_dict = {}
            for i in explanations.explainers[idx].pruned_rule:
                if i[1]:
                    chirps_dict[i[0]] = [['<=',float(i[2])]]
                else:
                    chirps_dict[i[0]] = [['>',float(i[2])]]
            return chirps_dict, 0, chte

        #LionForests
        def lf_rule(instance):
            temp = lf.explain(instance)[5]
            rule = {}
            for key,value in temp.items():
                rule[key] = [['<=',value[1]],['>',value[0]]]
            return rule

        return {'gs':gt.rule,'ls':lt.rule,'an':anchors_method,'lf':lf_rule, 'df': def_cov, 'ch':chirps_method}
    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf, 'classification')
    if tech:
        return interpretation, lf
    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] <= rule[feature][1][1]:#THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
        return 1
    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] < rule[feature][1][1]:#THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
        return 1

    #gt = GlobalSurrogateTree(train,lf.model.predict(train),feature_names,'c')
    #lt = LocalSurrogateTree(train,lf.model.predict(train),feature_names,'c',150)

    #loo = LeaveOneOut()
    #loo.get_n_splits(test)
    rule_generator = interpretation
    full_coverage = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    rule_length = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    time_response = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}

    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions
    
    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)   

    for tesd_ind in range(len(test)):
        for name, method in rule_generator.items():
            if name == 'ch':
                rule, op, te = method(tesd_ind)
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[tesd_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
            elif name == 'df':
                #FIX RULE LENGTH!
                rule, cover, predictions, te = method(np.concatenate((x_test_temp[tesd_ind:tesd_ind+1],x_test_temp)))
                f_precision[name].append(precision_score(predictions,y_test_temp,average='micro'))
                full_coverage[name] = full_coverage[name] + cover
            elif name == 'lf':
                ts = time.time()
                rule = method(x_test_temp_lf[tesd_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp_lf:
                    res = rule_cov_LF(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[tesd_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp_lf)
            else:
                ts = time.time()
                rule, prediction = method(x_test_temp[tesd_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0 
                for i in x_test_temp:
                    if name == 'anchors':
                        res = rule_cov(i, feature_names, rule)
                    else:    
                        res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1 and name=='anchors':
                        precision.append([y_test_temp[tesd_ind], y_test_temp[co]])
                    elif res == 1:
                        precision.append([prediction, y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
            time_response[name].append(te)
            rule_length[name] = rule_length[name] + len(rule)
    return rule_generator, full_coverage, rule_length, f_precision, time_response

In [5]:
total_results = []
kf = KFold(n_splits=10)
folds = 0
test_size = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    test_size.append(len(X_test))
    print('# of Fold: ' + str(folds+1) + ', size of test: ' + str(len(X_test)))    
    results = measure(X_train, X_test, y_train, y_test, feature_names, class_names)
    folds=folds+1
    total_results.append(results)

# of Fold: 1, size of test: 138
len self.path_detail
138
Working on CHIRPS for instance 0 of 138
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.1
found 9 patterns from 500 trees for batch_idx 0
start score sort for batch_idx 0 (9) patterns
start merge rule for batch_idx 0 (9) patterns
[('variance', False, 0.04431), ('curtosis', False, -0.92493)]
0.8970840480274442 0.42502617384735675 1.0 1.0
merge complete for batch_idx 0 (9) patterns
start get explainer for batch_idx 0
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.1
found 9 patterns from 500 trees for batch_idx 1
start score sort for batch_idx 1 (9) patterns
start merge rule for batch_idx 1 (9) patterns
[('variance', False, 0.04431), ('curtosis', False, -0.91968)]
0.8994800693240901 0.4221609658480989 1.0 0.9999999999999999
merge complete for batch_idx 1 (9) patterns
start get explainer for batch_idx 1
as_chirps for batch_idx 2
start mining for batch_idx 2 with support = 0.1
reduced 5

[('variance', False, 0.07938)]
0.8741379310344828 0.41287355712525015 0.31170137832796063 0.42408015201243726
merge complete for batch_idx 17 (49) patterns
start get explainer for batch_idx 17
as_chirps for batch_idx 18
start mining for batch_idx 18 with support = 0.1
reduced 3 patterns out of 52 by numeric redundancy
found 49 patterns from 493 trees for batch_idx 18
start score sort for batch_idx 18 (49) patterns
start merge rule for batch_idx 18 (49) patterns
[('curtosis', False, -0.70454), ('variance', False, 0.0798)]
0.9743589743589743 0.3095484845673695 0.39572731782317394 0.4946570817952206
merge complete for batch_idx 18 (49) patterns
start get explainer for batch_idx 18
as_chirps for batch_idx 19
start mining for batch_idx 19 with support = 0.1
found 8 patterns from 500 trees for batch_idx 19
start score sort for batch_idx 19 (8) patterns
start merge rule for batch_idx 19 (8) patterns
[('skew', False, 0.43128), ('variance', False, -0.49575)]
0.9907120743034056 0.259073379540665

[('curtosis', False, -0.83592), ('variance', False, 0.04454)]
0.9080459770114943 0.38867169381237 0.9999999999999999 0.9999999999999999
merge complete for batch_idx 35 (8) patterns
start get explainer for batch_idx 35
as_chirps for batch_idx 36
start mining for batch_idx 36 with support = 0.1
found 8 patterns from 500 trees for batch_idx 36
start score sort for batch_idx 36 (8) patterns
start merge rule for batch_idx 36 (8) patterns
[('curtosis', False, -0.83592), ('variance', False, 0.04454)]
0.9080459770114943 0.38867169381237 1.0 0.9999999999999999
merge complete for batch_idx 36 (8) patterns
start get explainer for batch_idx 36
as_chirps for batch_idx 37
start mining for batch_idx 37 with support = 0.1
reduced 2 patterns out of 15 by numeric redundancy
found 13 patterns from 500 trees for batch_idx 37
start score sort for batch_idx 37 (13) patterns
start merge rule for batch_idx 37 (13) patterns
[('variance', False, 0.08179), ('curtosis', False, -0.55183)]
0.9835526315789473 0.2429

[('curtosis', False, -0.69659), ('variance', False, 0.04454)]
0.9495192307692307 0.3242561988947347 0.9999999999999998 0.9999999999999998
merge complete for batch_idx 53 (10) patterns
start get explainer for batch_idx 53
as_chirps for batch_idx 54
start mining for batch_idx 54 with support = 0.1
found 9 patterns from 500 trees for batch_idx 54
start score sort for batch_idx 54 (9) patterns
start merge rule for batch_idx 54 (9) patterns
[('variance', False, 0.04431), ('curtosis', False, -0.91968)]
0.8994800693240901 0.4221609658480989 1.0 0.9999999999999999
merge complete for batch_idx 54 (9) patterns
start get explainer for batch_idx 54
Working on CHIRPS for instance 55 of 138
as_chirps for batch_idx 55
start mining for batch_idx 55 with support = 0.1
reduced 2 patterns out of 15 by numeric redundancy
found 13 patterns from 500 trees for batch_idx 55
start score sort for batch_idx 55 (13) patterns
start merge rule for batch_idx 55 (13) patterns
[('variance', False, 0.08179), ('curtosis

start merge rule for batch_idx 71 (13) patterns
[('variance', False, 0.08185), ('curtosis', False, -0.55189)]
0.9835526315789473 0.2429383622460474 0.7791329229143187 0.810137839039573
merge complete for batch_idx 71 (13) patterns
start get explainer for batch_idx 71
as_chirps for batch_idx 72
start mining for batch_idx 72 with support = 0.1
found 13 patterns from 500 trees for batch_idx 72
start score sort for batch_idx 72 (13) patterns
start merge rule for batch_idx 72 (13) patterns
[('variance', False, 0.04431), ('curtosis', False, -0.92505)]
0.8970840480274442 0.42502617384735675 0.9277248687483401 0.8649940262843487
merge complete for batch_idx 72 (13) patterns
start get explainer for batch_idx 72
as_chirps for batch_idx 73
start mining for batch_idx 73 with support = 0.1
reduced 5 patterns out of 18 by numeric redundancy
found 13 patterns from 500 trees for batch_idx 73
start score sort for batch_idx 73 (13) patterns
start merge rule for batch_idx 73 (13) patterns
[('variance', F

...

[('skew', True, 0.46859), ('variance', True, -0.2325)]
0.9770642201834863 0.17378805032246578 0.45546016861123106 0.5855599772598067
merge complete for batch_idx 67 (30) patterns
start get explainer for batch_idx 67
as_chirps for batch_idx 68
start mining for batch_idx 68 with support = 0.1
reduced 15 patterns out of 52 by numeric redundancy
found 37 patterns from 500 trees for batch_idx 68
start score sort for batch_idx 68 (37) patterns
start merge rule for batch_idx 68 (37) patterns
[('skew', True, -0.25824), ('variance', True, -0.04158)]
0.9793103448275862 0.11535729883522233 0.309237697044694 0.32874828060522693
merge complete for batch_idx 68 (37) patterns
start get explainer for batch_idx 68
as_chirps for batch_idx 69
start mining for batch_idx 69 with support = 0.1
reduced 15 patterns out of 47 by numeric redundancy
found 32 patterns from 497 trees for batch_idx 69
start score sort for batch_idx 69 (32) patterns
start merge rule for batch_idx 69 (32) patterns
[('skew', True, -0.

[('skew', True, 0.42458), ('variance', True, 0.15688)]
0.8556701030927835 0.35532927124012564 0.2665189845259319 0.2648401826484018
merge complete for batch_idx 84 (75) patterns
start get explainer for batch_idx 84
Working on CHIRPS for instance 85 of 137
as_chirps for batch_idx 85
start mining for batch_idx 85 with support = 0.1
reduced 9 patterns out of 42 by numeric redundancy
found 33 patterns from 500 trees for batch_idx 85
start score sort for batch_idx 85 (33) patterns
start merge rule for batch_idx 85 (33) patterns
[('skew', True, 0.0367), ('variance', True, -0.03455)]
0.9724770642201835 0.17355908055919506 0.3452504669633607 0.47393580696562704
merge complete for batch_idx 85 (33) patterns
start get explainer for batch_idx 85
as_chirps for batch_idx 86
start mining for batch_idx 86 with support = 0.1
reduced 16 patterns out of 55 by numeric redundancy
found 39 patterns from 500 trees for batch_idx 86
start score sort for batch_idx 86 (39) patterns
start merge rule for batch_id

[('skew', True, -0.30883), ('variance', True, -0.04126)]
0.9788732394366197 0.11293721564287501 0.32461229833227756 0.35451747238819475
merge complete for batch_idx 101 (39) patterns
start get explainer for batch_idx 101
as_chirps for batch_idx 102
start mining for batch_idx 102 with support = 0.1
reduced 2 patterns out of 51 by numeric redundancy
found 49 patterns from 400 trees for batch_idx 102
start score sort for batch_idx 102 (49) patterns
start merge rule for batch_idx 102 (49) patterns
[('skew', True, 0.39767), ('variance', True, 0.06368)]
0.912621359223301 0.316402291393705 0.6323068541885928 0.5899133978604179
merge complete for batch_idx 102 (49) patterns
start get explainer for batch_idx 102
as_chirps for batch_idx 103
start mining for batch_idx 103 with support = 0.1
found 45 patterns from 396 trees for batch_idx 103
start score sort for batch_idx 103 (45) patterns
start merge rule for batch_idx 103 (45) patterns
[('curtosis', True, -0.71495), ('skew', True, 0.48333), ('va

[('skew', True, 0.43679), ('variance', True, 0.07707)]
0.9035294117647059 0.3241925635709107 0.5403679889802708 0.4671635777598713
merge complete for batch_idx 118 (50) patterns
start get explainer for batch_idx 118
as_chirps for batch_idx 119
start mining for batch_idx 119 with support = 0.1
reduced 4 patterns out of 27 by numeric redundancy
found 23 patterns from 500 trees for batch_idx 119
start score sort for batch_idx 119 (23) patterns
start merge rule for batch_idx 119 (23) patterns
[('variance', True, -0.03423), ('skew', True, 0.60901)]
0.9421052631578948 0.29731723760701156 0.4944887546022084 0.6255304101838757
merge complete for batch_idx 119 (23) patterns
start get explainer for batch_idx 119
Working on CHIRPS for instance 120 of 137
as_chirps for batch_idx 120
start mining for batch_idx 120 with support = 0.1
found 16 patterns from 487 trees for batch_idx 120
start score sort for batch_idx 120 (16) patterns
start merge rule for batch_idx 120 (16) patterns
[('variance', True,

[('skew', True, -0.30883), ('variance', True, -0.04126)]
0.9788732394366197 0.11293721564287501 0.3246123242172071 0.35451747238819475
merge complete for batch_idx 135 (39) patterns
start get explainer for batch_idx 135
as_chirps for batch_idx 136
start mining for batch_idx 136 with support = 0.1
reduced 6 patterns out of 37 by numeric redundancy
found 31 patterns from 500 trees for batch_idx 136
start score sort for batch_idx 136 (31) patterns
start merge rule for batch_idx 136 (31) patterns
[('skew', True, 0.46706), ('variance', True, -0.23275)]
0.9769585253456221 0.1729834760154173 0.44509349464280634 0.547070258261594
merge complete for batch_idx 136 (31) patterns
start get explainer for batch_idx 136


In [6]:
full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
f_time = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
k = 0
for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/test_size[k])
        rule_length[name].append(i[2][name]/test_size[k])
        f_precision[name].append(np.array(i[3][name]).mean())
        f_time[name].append(np.array(i[4][name]).mean())
    k = + 1
for name, method in i[0].items():
    print(name,np.array(full_coverage[name]).mean(),',',
          np.array(rule_length[name]).mean(),',',
          np.array(f_precision[name]).mean(),',',
          np.array(f_time[name]).mean())

gs 0.14839779412102896 , 2.9681159420289855 , 0.9977272727272727 , 0.00040115909476538987
ls 0.5169207076351807 , 1.518840579710145 , 0.9658126398415066 , 2.149025497154799
an 0.299927789199656 , 2.1072463768115943 , 0.5541525303241762 , 3.3272864982272963
lf 0.06622496960553892 , 2.744202898550724 , 1.0 , 0.9793111042611677
df 0.9833333333333345 , 6.171739130434783 , 0.8966042526182166 , 0.20528782182656355
ch 0.5523581579040162 , 1.7608695652173914 , 0.9929425405727924 , 0.2786853864875708


In [6]:
full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
f_time = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
k = 0
for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/test_size[k])
        rule_length[name].append(i[2][name]/test_size[k])
        f_precision[name].append(np.array(i[3][name]).mean())
        f_time[name].append(np.array(i[4][name]).mean())
    k = + 1
for name, method in i[0].items():
    print(name,np.array(full_coverage[name]).mean(),',',
          np.array(rule_length[name]).mean(),',',
          np.array(f_precision[name]).mean(),',',
          np.array(f_time[name]).mean())

gs 0.07241573202532522 , 0.032417517333040435 2.9681159420289855 , 0.41445474640785074 1.0 , 0.0 0.0003684014476040463 , 0.00012943900043643826
ls 0.4010463066487084 , 0.14288309288036027 1.518840579710145 , 0.39157532217219393 0.7569185607087776 , 0.037830481972265226 2.118533880174898 , 0.02485086465130368
an 0.15071292262844369 , 0.060057505486948964 2.1072463768115943 , 0.6024158463402711 0.5531118934268553 , 0.45418812576442175 3.4690915730382152 , 1.2438905731700243
lf 0.031393945774112586 , 0.02070102791663443 2.744202898550724 , 0.26407153276553547 1.0 , 0.0 0.9968228860833612 , 0.07844917608607267
df 0.9942028985507246 , 0.0028985507246376938 6.171739130434783 , 1.797028545739577 0.9229800064304884 , 0.01897839463011047 8.222050204247278e-05 , 9.908882600092347e-05
ch 0.30591338706528476 , 0.10163114167023175 1.7608695652173914 , 0.1665090622184936 0.908153799158834 , 0.03303546230676742 0.2685065125594325 , 0.03835483807236268


# Conclusiveness Check:

Let's test a few examples!

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10, stratify=y, random_state=7)
techniques = measure(X_train, X_test, y_train, y_test, feature_names, class_names, True)

We prepare our explanation techniques. They are trained in almost the whole train set

In [482]:
lf = techniques[1]
techniques = techniques[0]
X_test_scaled = lf.utilizer.transform(X_test)
X_train_scaled = lf.utilizer.transform(X_train)

We select the instance 9

In [487]:
temp_id = 9
temp = X_test[temp_id].copy()
temp_scaled = lf.utilizer.transform([X_test[temp_id],X_test[temp_id]])[0]
temp, temp_scaled

(array([ 3.583  , -3.7971 ,  3.4391 , -0.12501]),
 array([ 0.53244056, -0.25342473, -0.24826502,  0.53180938]))

Know let's generate 

In [488]:
techniques['ls'](temp_scaled) #faulty for for id 9, temp2[2] = -5 for id 8, id 5, curt id 4

({}, 0)

Local Surrogate said 0 without giving any info

In [489]:
techniques['an'](temp_scaled) #faulty for temp2[2] = -5 for id 9

({'variance': [['>', 0.42]]}, 0)

Anchors said if variance > 0.42

In [490]:
techniques['ch'](temp_id) #faulty for temp2[2] = -5 for id 9 and id 8 amd od 4, temp2[1] = -13.7731 for id 5

({'variance': [['>', 0.23071]]}, 0)

CHIRPS Said 0 variance > 0.23071

In [491]:
techniques['gs'](temp_scaled)

({'skew': [['<=', 0.66949972230517]],
  'variance': [['>', 0.1280646916062096]],
  'curtosis': [['>', -0.6443884608512067]]},
 0)

The Global Suroggate said 0 because variance > 0.1280646916062096 and skew <= 0.66949972230517 and curtosis > -0.6443884608512067

In [492]:
techniques['lf'](temp)

{'variance': [['<=', 6.8248], ['>', 2.3942999407537267]],
 'curtosis': [['<=', 17.9274], ['>', -0.8297101872697472]]}

Finally, LF said 0 if 0.36099632 < variance < 1 and -0.61605188 < curtosis < 1. Those values are transformed, because LF provides ranges for the real values of an instance and not for the scaled values.

In [498]:
temp2 = temp.copy()
temp2[2] = -5 #Let's change the curtosis value and put it to the min value. Scaled value is -1 
temp2_scaled = lf.utilizer.transform([temp2,temp2])[0]
lf.model.predict([temp_scaled,temp2_scaled])

array([0, 1], dtype=int64)

Local surrogate did not gave any valuable info as well. Anchors and CHIRPS did not provided any condition for the curtosis feature. And when we modified it's value the prediction changed. Therefore, we can say that the two rules were inconclusive. Global Surrogate provided a conclusive answer, like LF. 

In [500]:
temp_id = 6
temp = X_test[temp_id].copy()
temp_scaled = lf.utilizer.transform([X_test[temp_id],X_test[temp_id]])[0]
temp, temp_scaled

(array([ 3.5156, 10.1891, -4.2759, -4.978 ]),
 array([ 0.52271957,  0.79326241, -0.91296444, -0.35073697]))

In [485]:
techniques['df'](np.array([temp_scaled,temp_scaled, temp_scaled, temp_scaled]))

[Rule  1]
y = 0 when
	 skew > 0.337126
	 curtosis > -0.974536

[Rule  2]
y = 0 when
	 variance > 0.078377
	 skew <= 0.400540
	 curtosis > -0.726306
	 entropy > 0.160262

[Rule  3]
y = 1 when
	 variance <= 0.118145
	 skew <= 0.301429
	 curtosis > -0.484685

[Rule  4]
y = 1 when
	 -0.645371 < variance <= 0.308764
	 -0.135811 < skew <= 0.420517
	 curtosis <= -0.274181
	 entropy <= 0.777904

[Otherwise]
y = 0



([0, 0], 1.0, array([0, 0, 0], dtype=int64))

In [484]:
techniques['lf'](temp)

{'variance': [['<=', 6.8248], ['>', 2.8434999517112955]],
 'curtosis': [['<=', -4.02399996997267], ['>', -4.364150089024007]]}

In [505]:
temp2 = temp.copy()
temp2[0] = -7 #Let's change the curtosis value and put it to the min value. Scaled value is -1 
temp2_scaled = lf.utilizer.transform([temp2,temp2])[0]
lf.model.predict([temp_scaled,temp2_scaled])

array([0, 1], dtype=int64)

Let's try one more example to prove that GS is also inconclusive

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10, stratify=y, random_state=77)
techniques = measure(X_train, X_test, y_train, y_test, feature_names, class_names, True)

In [465]:
lf = techniques[1]
techniques = techniques[0]
X_test_scaled = lf.utilizer.transform(X_test)
X_train_scaled = lf.utilizer.transform(X_train)

We select a random instance again

In [466]:
temp_id = 0
temp = X_test[temp_id].copy()
temp_scaled = lf.utilizer.transform([X_test[temp_id],X_test[temp_id]])[0]
temp, temp_scaled

(array([-1.6936,  2.7852, -2.1835, -1.9276]),
 array([-0.22859471,  0.23917574, -0.73269003,  0.2039972 ]))

In [467]:
techniques['gs'](temp_scaled)

({'variance': [['<=', 0.05603344863243409], ['>', -0.25483223672374644]],
  'skew': [['<=', 0.32315588433361764], ['>', 0.22769872546014208]]},
 1)

GS said 1 if -0.2548322 < Variance <= 0.05603344 and 0.2276987254 < skew <= 0.32315588433361764

In [468]:
techniques['lf'](temp)

{'variance': [['<=', -0.966239995934629], ['>', -1.698349986315147]],
 'skew': [['<=', 3.9254000015519557], ['>', -13.773100000000001]],
 'curtosis': [['<=', -0.2843348647490144], ['>', -5.286099999999999]]}

And LF said 0 because: -0.229 < variance <= -0.12368878 and -1 < skew <= 0.32 and -1 < curtosis <= -0.56906411

In [480]:
temp2 = temp.copy()

temp2[2] = 17.9274 

temp2_scaled = lf.utilizer.transform([temp2,temp2])[0]
lf.model.predict([temp_scaled,temp2_scaled])

array([1, 0], dtype=int64)

We changed the value of curtosis, which was not iuncluded in GS's rule and the prediction changed. GS's rule therefore it was not conclusive